In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_data = pd.read_csv('../data/archive/cross-val/eyes/eyesLabels.csv')

In [4]:
Y = train_data[['label']]

In [5]:
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

In [6]:
idg  = ImageDataGenerator(rescale=1./255)

In [19]:
#model = keras.models.load_model("base_eyes_model.h5")
#model = keras.models.load_model("mobile_eyes_model.h5")
model = keras.models.load_model("inception_eyes_model.h5")

c:\Users\xuton\anaconda3\lib\site-packages\keras\layers\core\lambda_layer.py:303: UserWarning: tensorflow.python.keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1, 1, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 1537      
                                                                 
Total params: 54,338,273
Trainable params: 54,277,729
Non-trainable params: 60,544
_________________________________________________________________


In [21]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

for train_index, val_index in skf.split(np.zeros(len(Y)),Y):
	validation_data = train_data.iloc[val_index]
	
	# cropped mouth is (80, 80) else (224, 224) 
	valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = '../data/archive/', target_size=(80, 80),
							x_col = "filename", y_col = "label",
							class_mode = "binary", shuffle = True)
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])

Found 291 validated image filenames belonging to 2 classes.
10/10 [==============================] - 7s 144ms/step - loss: 4.2734e-04 - accuracy: 1.0000
Found 291 validated image filenames belonging to 2 classes.
10/10 [==============================] - 2s 149ms/step - loss: 6.1458e-04 - accuracy: 1.0000
Found 290 validated image filenames belonging to 2 classes.
10/10 [==============================] - 2s 174ms/step - loss: 0.0028 - accuracy: 1.0000
Found 290 validated image filenames belonging to 2 classes.
10/10 [==============================] - 2s 157ms/step - loss: 2.0421 - accuracy: 0.9966
Found 290 validated image filenames belonging to 2 classes.
10/10 [==============================] - 1s 103ms/step - loss: 0.0016 - accuracy: 1.0000


In [13]:
np.mean(VALIDATION_ACCURACY) # base eyes

0.984848928451538

In [18]:
np.mean(VALIDATION_ACCURACY) # mobile eyes

0.9965564608573914

In [22]:
np.mean(VALIDATION_ACCURACY) # inception eyes

0.9993103504180908

In [23]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
test_generator = test_datagen.flow_from_directory(
        '../data/test/yourfolderforcroppedeyes',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 45 images belonging to 2 classes.


In [25]:
model.evaluate(test_generator)

3/3 [==============================] - 1s 308ms/step - loss: 4.5686e-04 - accuracy: 1.0000


[0.0004568615404423326, 1.0]